In [1]:
import datetime
import pandas as pd

from functions import Util

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'
path_data = './data'
path_csv = path_data + '/csv'
file_name_out_r = path_csv + '/covid19-regioni.csv'
file_name_pop_r = path_csv + '/popolazione-regioni.csv'

In [2]:
# Define today, yesterday and day before yesterday dates
date_tday = datetime.date.today()
date_yday = Util.get_previous_date(date_tday)
date_bf_yday = Util.get_previous_date(date_yday)
date_tday, date_yday, date_bf_yday

(datetime.date(2020, 10, 9),
 datetime.date(2020, 10, 8),
 datetime.date(2020, 10, 7))

In [3]:
# Today url for regions file
url_tday = Util.get_url_date(level, date_tday)
# Yesterday url for regions file
url_yday = Util.get_url_date(level, date_yday)
# Day before yesterday url for regions file
url_bf_yday = Util.get_url_date(level, date_bf_yday)

url_tday, url_yday, url_bf_yday

('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201009.csv',
 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201008.csv',
 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni-20201007.csv')

In [4]:
# Get csv files
try:
    # today and yesterday if today present
    df_raw_r_1 = pd.read_csv(url_tday)
    print('Presenti dati odierni')
    df_raw_r_0 = pd.read_csv(url_yday)
    date_last = date_tday
except Exception as e:
    # yesterday and day before yesterday otherwise
    df_raw_r_1 = pd.read_csv(url_yday)
    df_raw_r_0 = pd.read_csv(url_bf_yday)
    date_last = date_yday
    print('Dati di ieri')

Dati di ieri


In [5]:
# State S
df_raw_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-10-08T17:00:00,ITA,1,Piemonte,45.073274,7.680687,311,16,327,3652,...,255,336,29043,4172,24168,13026,37194,771156,459191,NaN
1,2020-10-08T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,4,1,5,117,...,15,18,1116,146,1244,140,1384,30644,21086,NaN
2,2020-10-08T17:00:00,ITA,3,Lombardia,45.466794,9.190347,361,41,402,10004,...,529,683,82484,16979,97091,12778,109869,2254865,1408098,NaN
3,2020-10-08T17:00:00,ITA,5,Veneto,45.434905,12.338452,213,19,232,4851,...,329,491,22620,2206,21837,8072,29909,2023146,790836,NaN
4,2020-10-08T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,23,6,29,991,...,75,110,3769,355,4732,412,5144,439370,194340,NaN
5,2020-10-08T17:00:00,ITA,7,Liguria,44.411493,8.932699,195,28,223,1932,...,123,152,10705,1614,10066,4408,14474,336672,179543,NaN
6,2020-10-08T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,232,13,245,5179,...,129,184,26722,4492,31698,4940,36638,1257805,704510,NaN
7,2020-10-08T17:00:00,ITA,9,Toscana,43.769231,11.255889,132,27,159,4590,...,319,339,10693,1170,12785,3827,16612,806096,541159,NaN
8,2020-10-08T17:00:00,ITA,10,Umbria,43.106758,12.388247,50,8,58,805,...,55,84,1956,87,1770,1136,2906,221146,131965,NaN
9,2020-10-08T17:00:00,ITA,11,Marche,43.616760,13.518875,34,5,39,1068,...,55,66,6251,992,8350,0,8350,261317,153648,NaN


In [6]:
# State S - 1
df_raw_r_0.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-10-07T17:00:00,ITA,1,Piemonte,45.073274,7.680687,297,15,312,3412,...,142,287,28962,4172,24075,12783,36858,763688,456021,NaN
1,2020-10-07T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,4,1,5,102,...,14,15,1113,146,1227,139,1366,30395,20931,NaN
2,2020-10-07T17:00:00,ITA,3,Lombardia,45.466794,9.190347,339,40,379,9498,...,368,520,82331,16978,96683,12503,109186,2232796,1395667,NaN
3,2020-10-07T17:00:00,ITA,5,Veneto,45.434905,12.338452,214,16,230,4524,...,228,375,22464,2200,21700,7718,29418,2006188,784296,NaN
4,2020-10-07T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,26,6,32,913,...,37,72,3734,355,4632,402,5034,434417,192089,NaN
5,2020-10-07T17:00:00,ITA,7,Liguria,44.411493,8.932699,196,24,220,1812,...,107,176,10676,1614,10046,4276,14322,332815,177511,NaN
6,2020-10-07T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,223,14,237,5058,...,141,193,26668,4491,31585,4869,36454,1246110,698835,NaN
7,2020-10-07T17:00:00,ITA,9,Toscana,43.769231,11.255889,123,28,151,4279,...,231,300,10673,1170,12593,3680,16273,797046,535058,NaN
8,2020-10-07T17:00:00,ITA,10,Umbria,43.106758,12.388247,44,8,52,756,...,60,65,1928,86,1742,1080,2822,218766,135552,NaN
9,2020-10-07T17:00:00,ITA,11,Marche,43.616760,13.518875,31,5,36,1016,...,70,84,6240,992,8284,0,8284,259255,152490,NaN


In [7]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Util.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-10-08T17:00:00,ITA,1,Piemonte,45.073274,7.680687,311,16,327,3652,...,255,336,29043,4172,24168,13026,37194,771156,459191,NaN
1,2020-10-08T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,4,1,5,117,...,15,18,1116,146,1244,140,1384,30644,21086,NaN
2,2020-10-08T17:00:00,ITA,3,Lombardia,45.466794,9.190347,361,41,402,10004,...,529,683,82484,16979,97091,12778,109869,2254865,1408098,NaN
3,2020-10-08T17:00:00,ITA,4,Trentino-Alto Adige,46.382635,11.204676,48,1,49,1367,...,100,128,8044,698,7623,2535,10158,431943,201078,NaN
4,2020-10-08T17:00:00,ITA,5,Veneto,45.434905,12.338452,213,19,232,4851,...,329,491,22620,2206,21837,8072,29909,2023146,790836,NaN
5,2020-10-08T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,23,6,29,991,...,75,110,3769,355,4732,412,5144,439370,194340,NaN
6,2020-10-08T17:00:00,ITA,7,Liguria,44.411493,8.932699,195,28,223,1932,...,123,152,10705,1614,10066,4408,14474,336672,179543,NaN
7,2020-10-08T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,232,13,245,5179,...,129,184,26722,4492,31698,4940,36638,1257805,704510,NaN
8,2020-10-08T17:00:00,ITA,9,Toscana,43.769231,11.255889,132,27,159,4590,...,319,339,10693,1170,12785,3827,16612,806096,541159,NaN
9,2020-10-08T17:00:00,ITA,10,Umbria,43.106758,12.388247,50,8,58,805,...,55,84,1956,87,1770,1136,2906,221146,131965,NaN


In [8]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Util.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,11,2020-10-07T17:00:00,ITA,1,Piemonte,45.073274,7.680687,297,15,312,...,142,287,28962,4172,24075,12783,36858,763688,456021,NaN
1,17,2020-10-07T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,4,1,5,...,14,15,1113,146,1227,139,1366,30395,20931,NaN
2,8,2020-10-07T17:00:00,ITA,3,Lombardia,45.466794,9.190347,339,40,379,...,368,520,82331,16978,96683,12503,109186,2232796,1395667,NaN
3,19,2020-10-07T17:00:00,ITA,4,Trentino-Alto Adige,46.382635,11.204676,39,1,40,...,23,87,8016,698,7541,2489,10030,427538,199281,NaN
4,18,2020-10-07T17:00:00,ITA,5,Veneto,45.434905,12.338452,214,16,230,...,228,375,22464,2200,21700,7718,29418,2006188,784296,NaN
5,5,2020-10-07T17:00:00,ITA,6,Friuli Venezia Giulia,45.649435,13.768136,26,6,32,...,37,72,3734,355,4632,402,5034,434417,192089,NaN
6,7,2020-10-07T17:00:00,ITA,7,Liguria,44.411493,8.932699,196,24,220,...,107,176,10676,1614,10046,4276,14322,332815,177511,NaN
7,4,2020-10-07T17:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,223,14,237,...,141,193,26668,4491,31585,4869,36454,1246110,698835,NaN
8,15,2020-10-07T17:00:00,ITA,9,Toscana,43.769231,11.255889,123,28,151,...,231,300,10673,1170,12593,3680,16273,797046,535058,NaN
9,16,2020-10-07T17:00:00,ITA,10,Umbria,43.106758,12.388247,44,8,52,...,60,65,1928,86,1742,1080,2822,218766,135552,NaN


In [9]:
# Select columns
cols_sel = ['data', 'codice_regione', 'denominazione_regione',
            'ricoverati_con_sintomi', 'terapia_intensiva',
            'totale_ospedalizzati', 'isolamento_domiciliare',
            'totale_positivi', 'variazione_totale_positivi',
            'nuovi_positivi', 'dimessi_guariti', 'deceduti',
            'totale_casi', 'tamponi', 'casi_testati']
# df_r_1.columns

In [10]:
df_rf_1 = df_r_1[cols_sel]
df_rf_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,totale_casi,tamponi,casi_testati
0,2020-10-08T17:00:00,1,Piemonte,311,16,327,3652,3979,255,336,29043,4172,37194,771156,459191
1,2020-10-08T17:00:00,2,Valle d'Aosta,4,1,5,117,122,15,18,1116,146,1384,30644,21086
2,2020-10-08T17:00:00,3,Lombardia,361,41,402,10004,10406,529,683,82484,16979,109869,2254865,1408098
3,2020-10-08T17:00:00,4,Trentino-Alto Adige,48,1,49,1367,1416,100,128,8044,698,10158,431943,201078
4,2020-10-08T17:00:00,5,Veneto,213,19,232,4851,5083,329,491,22620,2206,29909,2023146,790836
5,2020-10-08T17:00:00,6,Friuli Venezia Giulia,23,6,29,991,1020,75,110,3769,355,5144,439370,194340
6,2020-10-08T17:00:00,7,Liguria,195,28,223,1932,2155,123,152,10705,1614,14474,336672,179543
7,2020-10-08T17:00:00,8,Emilia-Romagna,232,13,245,5179,5424,129,184,26722,4492,36638,1257805,704510
8,2020-10-08T17:00:00,9,Toscana,132,27,159,4590,4749,319,339,10693,1170,16612,806096,541159
9,2020-10-08T17:00:00,10,Umbria,50,8,58,805,863,55,84,1956,87,2906,221146,131965


In [11]:
df_rf_0 = df_r_0[cols_sel]
df_rf_0.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,totale_casi,tamponi,casi_testati
0,2020-10-07T17:00:00,1,Piemonte,297,15,312,3412,3724,142,287,28962,4172,36858,763688,456021
1,2020-10-07T17:00:00,2,Valle d'Aosta,4,1,5,102,107,14,15,1113,146,1366,30395,20931
2,2020-10-07T17:00:00,3,Lombardia,339,40,379,9498,9877,368,520,82331,16978,109186,2232796,1395667
3,2020-10-07T17:00:00,4,Trentino-Alto Adige,39,1,40,1276,1316,23,87,8016,698,10030,427538,199281
4,2020-10-07T17:00:00,5,Veneto,214,16,230,4524,4754,228,375,22464,2200,29418,2006188,784296
5,2020-10-07T17:00:00,6,Friuli Venezia Giulia,26,6,32,913,945,37,72,3734,355,5034,434417,192089
6,2020-10-07T17:00:00,7,Liguria,196,24,220,1812,2032,107,176,10676,1614,14322,332815,177511
7,2020-10-07T17:00:00,8,Emilia-Romagna,223,14,237,5058,5295,141,193,26668,4491,36454,1246110,698835
8,2020-10-07T17:00:00,9,Toscana,123,28,151,4279,4430,231,300,10673,1170,16273,797046,535058
9,2020-10-07T17:00:00,10,Umbria,44,8,52,756,808,60,65,1928,86,2822,218766,135552


In [12]:
df_r = df_rf_1.copy()
df_pop_r = pd.read_csv(file_name_pop_r)
df_m_r = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,totale_casi,tamponi,casi_testati,popolazione
0,2020-10-08T17:00:00,1,Piemonte,311,16,327,3652,3979,255,336,29043,4172,37194,771156,459191,4341375
1,2020-10-08T17:00:00,2,Valle d'Aosta,4,1,5,117,122,15,18,1116,146,1384,30644,21086,125501
2,2020-10-08T17:00:00,3,Lombardia,361,41,402,10004,10406,529,683,82484,16979,109869,2254865,1408098,10103969
3,2020-10-08T17:00:00,4,Trentino-Alto Adige,48,1,49,1367,1416,100,128,8044,698,10158,431943,201078,1074819
4,2020-10-08T17:00:00,5,Veneto,213,19,232,4851,5083,329,491,22620,2206,29909,2023146,790836,4907704
5,2020-10-08T17:00:00,6,Friuli Venezia Giulia,23,6,29,991,1020,75,110,3769,355,5144,439370,194340,1211357
6,2020-10-08T17:00:00,7,Liguria,195,28,223,1932,2155,123,152,10705,1614,14474,336672,179543,1543127
7,2020-10-08T17:00:00,8,Emilia-Romagna,232,13,245,5179,5424,129,184,26722,4492,36638,1257805,704510,4467118
8,2020-10-08T17:00:00,9,Toscana,132,27,159,4590,4749,319,339,10693,1170,16612,806096,541159,3722729
9,2020-10-08T17:00:00,10,Umbria,50,8,58,805,863,55,84,1956,87,2906,221146,131965,880285


In [13]:
df_rf_1['ricoverati_con_sintomi'] - df_rf_0['ricoverati_con_sintomi']

0      2
1      0
2      2
3     28
4      9
5     -3
6     13
7     -1
8     22
9      3
10     2
11    14
12    27
13     1
14     1
15     9
16     6
17     0
18    -1
19     9
Name: ricoverati_con_sintomi, dtype: int64

In [14]:
# Add columns
df_r = df_m_r.copy()
df_r['variazione_ricoverati_con_sintomi'] = df_rf_1['ricoverati_con_sintomi'] - df_rf_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_rf_1['terapia_intensiva'] - df_rf_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_rf_1['totale_ospedalizzati'] - df_rf_0['totale_ospedalizzati']
df_r['variazione_isolamento_domiciliare'] = df_rf_1['isolamento_domiciliare'] - df_rf_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_rf_1['nuovi_positivi'] - df_rf_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_rf_1['tamponi'] - df_rf_0['tamponi']
df_r['nuovi_casi_testati'] = df_rf_1['casi_testati'] - df_rf_0['casi_testati']
df_r['nuovi_deceduti'] = df_rf_1['deceduti'] - df_rf_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_rf_1['dimessi_guariti'] - df_rf_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,variazione_totale_ospedalizzati,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-10-08T17:00:00,15,Campania,548,55,603,8347,8950,607,757,...,31,576,213,9925,7891,1,149,7.63,0.01308,0.28
1,2020-10-08T17:00:00,3,Lombardia,361,41,402,10004,10406,529,683,...,23,506,163,22069,12431,1,153,3.09,0.00676,1.09
2,2020-10-08T17:00:00,5,Veneto,213,19,232,4851,5083,329,491,...,2,327,116,16958,6540,6,156,2.90,0.01000,0.61
3,2020-10-08T17:00:00,12,Lazio,821,55,876,7976,8852,272,359,...,20,252,2,13333,9674,6,81,2.69,0.00612,0.32
4,2020-10-08T17:00:00,9,Toscana,132,27,159,4590,4749,319,339,...,8,311,39,9050,6101,0,20,3.75,0.00911,0.45
5,2020-10-08T17:00:00,1,Piemonte,311,16,327,3652,3979,255,336,...,15,240,49,7468,3170,0,81,4.50,0.00774,0.86
6,2020-10-08T17:00:00,19,Sicilia,376,33,409,3287,3696,147,259,...,4,143,46,7374,4806,3,109,3.51,0.00521,0.17
7,2020-10-08T17:00:00,16,Puglia,278,23,301,3033,3334,201,248,...,27,174,52,4754,3083,1,46,5.22,0.00619,0.22
8,2020-10-08T17:00:00,8,Emilia-Romagna,232,13,245,5179,5424,129,184,...,8,121,-9,11695,5675,1,54,1.57,0.00412,0.82
9,2020-10-08T17:00:00,7,Liguria,195,28,223,1932,2155,123,152,...,3,120,-24,3857,2032,0,29,3.94,0.00985,0.94


In [15]:
# Write pre-processed data
df_r.to_csv(file_name_out_r, index=False)